### Discovery Significance vs Mass

This notebook makes a p0 plot a scan of the discovery significance vs mass for the H->gg case


We read first the file with the workspace and we retrieve the needed objects

In [2]:
  // Check if example input file exists
  TFile *file = TFile::Open("HiggsModel.root");

  // get the workspace out of the file
  RooWorkspace* w = (RooWorkspace*) file->Get("w");

In [3]:
 // get the data out of the file
  RooAbsData* data2 = w->data("data2");

In [4]:
  // get the modelConfig (S+B) out of the file
  // and create the B model from the S+B model
  RooStats::ModelConfig*  sbModel = (RooStats::ModelConfig*) w->obj("ModelConfig");
  sbModel->SetName("S+B Model");      
  RooRealVar* poi = (RooRealVar*) sbModel->GetParametersOfInterest()->first();
  poi->setVal(200);  // set POI snapshot in S+B model for expected significance
  sbModel->SetSnapshot(*poi);
  RooStats::ModelConfig * bModel = (RooStats::ModelConfig*) sbModel->Clone();
  bModel->SetName("B Model");      
  poi->setVal(0);
  bModel->SetSnapshot( *poi  );

In [5]:
vector<double> masses;
vector<double> p0values;
vector<double> p0valuesExpected;
double massMin, massMax; 

###   Loop on the mass values

Perform a loop between the given values and perform an asymptotic hypothesis test to compute the p-value at each scanned mass point

In [ ]:
massMin = 112;
massMax = 158;


int npoints = 30;
for( double mass=massMin; mass<=massMax; mass += (massMax-massMin)/double(npoints) )
  {
      
      
     cout << endl << endl << "Running for mass: " << mass << endl << endl;
     w->var("mass")->setVal( mass );

     RooStats::AsymptoticCalculator ac(*data2, *sbModel, *bModel);
     ac.SetOneSidedDiscovery(true);  // for one-side discovery test                                      
     RooStats::AsymptoticCalculator::SetPrintLevel(-1);


     RooStats::HypoTestResult* asymCalcResult = ac.GetHypoTest();
 
     asymCalcResult->Print();
     
     masses.push_back( mass );
     p0values.push_back( asymCalcResult->NullPValue() );
  
     double expectedP0 = RooStats::AsymptoticCalculator::GetExpectedPValues(  asymCalcResult->NullPValue(),  asymCalcResult->AlternatePValue(), 0, false);
     p0valuesExpected.push_back( expectedP0 );
     std::cout << "expected p0 = " << expectedP0 << std::endl;

  }




Running for mass: 112

[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize - Find  best unconditional NLL on observed data
AsymptoticCalculator::EvaluateNLL  ........ using Minuit / Migrad with strategy  1 and tolerance 1
 **********
 **    1 **SET PRINT           0
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a1           8.62045e+00  8.79714e-01   -5.00000e+02  5.00000e+02
     2 a2          -2.02194e+00  3.32779e-01   -5.00000e+02  5.00000e+02
     3 nbackground   3.04986e+04  1.96318e+02    3.07700e+03  3.07700e+05
     4 nsignal      0.00000e+00  9.05627e+01    0.00000e+00  1.00000e+03
 MINUIT WARNING IN PARAM DEF
 ============== STARTING VALUE IS AT LIMIT.
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE4 IS AT ITS LOWER ALLOWED LIMIT.
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE4 BROUGHT BACK INSIDE LIMITS

[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set

Results HypoTestAsymptotic_result: 
 - Null p-value = 0.5
 - Significance = -0
 - CL_b: 0.5
 - CL_s+b: 0.998073
 - CL_s: 1.99615
expected p0 = 0.00192738


Running for mass: 128.867

[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set

Results HypoTestAsymptotic_result: 
 - Null p-value = 0.5
 - Significance = -0
 - CL_b: 0.5
 - CL_s+b: 0.998445
 - CL_s: 1.99689
expected p0 = 0.00155518


Running for mass: 130.4

[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set

Results HypoTestAsymptotic_result: 
 - Null p-value = 0.5
 - Significance = -0
 - CL_b: 0.5
 - CL_s+b: 0.998781
 - CL_s: 1.99756
expected p0 = 0.00121911


Running for mass: 131.933

[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set

Results HypoTestAsymptotic_result: 
 - Null p-value = 0.5
 - Significance = -0
 - CL_b: 0.5
 - CL_s+b: 0.999076
 - CL_s: 1.99815
expected p0 = 0.000923667


Running for mass

Create a plot with the obtained p-values

In [ ]:
  TGraph * graph1  = new TGraph(masses.size(),&masses[0],&p0values[0]);
  TGraph * graph2  = new TGraph(masses.size(),&masses[0],&p0valuesExpected[0]);

  graph1->SetMarkerStyle(20);
  graph1->Draw("APC");
  graph2->SetLineStyle(2);
  graph2->Draw("C");
  graph1->GetXaxis()->SetTitle("mass");
  graph1->GetYaxis()->SetTitle("p0 value");
  graph1->SetTitle("Significance vs Mass");
  graph1->SetMinimum(graph2->GetMinimum());
  graph1->SetLineColor(kBlue);
  graph2->SetLineColor(kRed);
  gPad->SetLogy(true);

In [ ]:
gPad->Draw()